In [91]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

--2025-02-10 23:49:19--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2025-02-10 23:49:19--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: 'creds.json'

     0K ..                                                    100% 15.2M=0s

2025-02-10 23:49:20 (15.2 MB/s) - 'creds.json' saved [2358/

In [92]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
client = gspread.authorize(creds)

In [93]:
installments = client.open("Installments")
sheet1_data = installments.worksheet("Лист1").get_all_records()
sheet2_data = installments.worksheet("Лист2").get_all_records()
sheet3_data = installments.worksheet("Лист3").get_all_records()

In [94]:
import datetime

until_date = datetime.datetime(2023, 3, 1).date()

filtered_studens = {
    int(el['student_id']): el['student_name']
    for el in sheet1_data
    if el['installment'] == 'Y'
}

filtered_dates = {
        int(el['student_id']): 
        int((until_date.toordinal() - datetime.datetime.strptime(str(el['last_payment_date']), '%d.%m.%Y').date().toordinal()) / 182)
    for el in sheet2_data
    if datetime.datetime.strptime(str(el['expected_payment_date']), '%d.%m.%Y').date() < until_date
}

filtered_payments = {
    int(el['student_id']): (int(el['one-time_payment']), int(el['left_to_pay']))
    for el in sheet3_data
}

with open('student_debt_report.txt', 'w', encoding='utf-8') as f:        
    for student_id, period in filtered_dates.items():
        amount = min(filtered_payments[student_id][0] * period, filtered_payments[student_id][1])
        f.write(f'Студент {filtered_studens[student_id]} - долг {amount} рублей\n')


In [95]:
!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2025-02-10 23:49:26--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2025-02-10 23:49:26--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: 'student_debt.txt'

     0K .......... .                                          100% 11.4M=0.001s

2025-02-10 23:49:26 (11.4 MB/s)

In [96]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
